下载 faiss 库和必要的工具包 nltk 分词器

In [ ]:
!pip install faiss-gpu
!pip install nltk
import nltk

nltk.download('punkt_tab')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

以下代码是简单的文本抄袭查询的实现模型，我们使用了简单的直接读取，随机生成了100句文本，人工查询了抄袭结果。

在这段代码中，我们只使用了显式抄袭的数据。使用不抄袭和隐式抄袭的数据的实验我们放在了下一个代码块中。

这段代码可以在colab上直接运行。

In [ ]:
import faiss
import numpy as np
import nltk
from sentence_transformers import SentenceTransformer

#查询目标改成全局参数
query_sentence = "I enjoy playing basketball with my friends. We play every weekend."
#随机生成100句文本，改成双句子
def generate_random_sentences():
    sentences = [
        "The cat sat on the mat. It looked very comfortable.",
        "A quick brown fox jumps over the lazy dog. This is a famous pangram.",
        "Artificial intelligence is the future of technology. Many companies are investing in it.",
        "The sun sets in the west. The sky turns orange and red.",
        "I love programming in Python. It is very versatile.",
        "Machine learning and data science are exciting fields. They are rapidly growing.",
        "The book is on the table. I will read it later.",
        "Dogs are loyal and friendly animals. They are often called man's best friend.",
        "The weather today is sunny and warm. It's perfect for a picnic.",
        "I enjoy playing basketball with my friends. We play every weekend.",
        "The stars twinkle in the night sky. They look beautiful.",
        "Coffee is a great way to start the day. It gives me energy.",
        "She reads a novel every evening. It helps her relax.",
        "The train arrives at the station on time. It is very reliable.",
        "Cooking is both an art and a science. It requires creativity.",
        "The children are playing in the park. They are having fun.",
        "Music has the power to heal the soul. It brings peace to the mind.",
        "I am learning to play the guitar. It's a challenging but rewarding hobby.",
        "Traveling helps us discover new perspectives. It broadens our horizons.",
        "Technology is evolving at a rapid pace. It impacts every aspect of our lives.",
        "The ocean waves crash against the shore. It's a soothing sound.",
        "He enjoys hiking in the mountains. The fresh air is invigorating.",
        "Education is the key to success. It empowers individuals and communities.",
        "The garden is full of blooming flowers. It smells delightful.",
        "She practices yoga every morning. It keeps her fit and focused.",
        "Reading books expands our knowledge. It also improves vocabulary.",
        "The road stretches far into the horizon. It's a scenic view.",
        "A healthy lifestyle promotes longevity. It also boosts happiness.",
        "Nature inspires creativity in people. It's a source of peace.",
        "The airplane soars high above the clouds. The view is breathtaking.",
        "Good communication strengthens relationships. It fosters understanding.",
        "The team worked hard to achieve their goal. Their efforts paid off.",
        "Rain brings freshness to the earth. It also makes everything green.",
        "Friendship makes life more enjoyable. It adds meaning to our lives.",
        "The artist paints a beautiful landscape. It's a masterpiece.",
        "The chef prepares a delicious meal. Everyone enjoys it.",
        "Learning new languages broadens horizons. It also improves cognitive skills.",
        "The library is a quiet place to study. It's filled with resources.",
        "The mountains are covered with snow. They look majestic.",
        "Exercise keeps the body and mind healthy. It reduces stress.",
        "Innovative ideas drive progress in society. They solve problems.",
        "The forest is home to diverse wildlife. It's a wonder of nature.",
        "Patience is a virtue that brings peace. It helps avoid conflicts.",
        "History teaches us valuable lessons. It shapes our future.",
        "He writes poetry to express his feelings. His words are heartfelt.",
        "The bridge connects the two islands. It is an engineering marvel.",
        "Adventure awaits those who seek it. It's a journey of discovery.",
        "The farmer works hard to grow crops. He feeds the nation.",
        "Kindness can change someone's day. It creates a ripple effect.",
        "The clock ticks steadily on the wall. Time never stops.",
        "Creativity fuels innovation in technology. It leads to breakthroughs.",
        "The river flows gently through the valley. It's a picturesque scene.",
        "Teamwork leads to great achievements. It builds strong bonds.",
        "Happiness comes from within. It's a state of mind.",
        "The museum showcases ancient artifacts. It's a window to the past.",
        "Dreams inspire us to reach for the stars. They give us hope.",
        "The scientist conducts groundbreaking research. Her work changes the world.",
        "Volunteering helps build a better community. It brings people together.",
        "Hard work paves the way to success. It instills discipline.",
        "The desert stretches endlessly under the sun. It's a harsh environment.",
        "The dancer moves gracefully across the stage. Her performance is mesmerizing.",
        "Cooking together strengthens family bonds. It creates memories.",
        "The eagle soars majestically in the sky. It's a symbol of freedom.",
        "The entrepreneur starts a new business venture. He takes calculated risks.",
        "Gardening is a relaxing and rewarding hobby. It connects us to nature.",
        "The child builds a sandcastle on the beach. It's a work of art.",
        "Books open doors to new worlds. They ignite imagination.",
        "A walk in the park refreshes the mind. It clears thoughts.",
        "The ship sails across the vast ocean. It's a journey of discovery.",
        "The sunset paints the sky with vibrant colors. It's a breathtaking sight.",
        "A smile can brighten someone's day. It's a universal language.",
        "Education empowers individuals and communities. It drives progress.",
        "He enjoys solving challenging puzzles. It sharpens his mind.",
        "The festival celebrates cultural diversity. It unites people.",
        "The teacher inspires students to learn. She shapes young minds.",
        "The wind whispers through the tall grass. It's a gentle melody.",
        "Innovation leads to breakthroughs in science. It changes the world.",
        "The mountain trail offers stunning views. It's worth the climb.",
        "The puppy plays joyfully in the garden. It's full of energy.",
        "Meditation brings clarity and peace of mind. It's a spiritual practice.",
        "The waterfall cascades down the rocks. It's a majestic sight.",
        "Collaboration fosters creative solutions. It brings out the best in everyone.",
        "The astronaut explores the mysteries of space. It's a dream come true.",
        "Laughter is the best medicine for the soul. It lightens the mood.",
        "The bridge glows under the city lights. It's a magical view.",
        "The explorer ventures into uncharted territories. He seeks the unknown.",
        "Photography captures moments in time. It's an art form.",
        "The city skyline shines brightly at night. It's a bustling metropolis.",
        "Imagination knows no bounds. It fuels creativity.",
        "The clock tower chimes every hour. I can hear it every hour.",
        "He enjoy playing basketball with my friends. They play every weekend.",
    ]
    return sentences

#文本向量化并使用改进的余弦相似度
def test_text_similarity():
    #加载更强大的句子嵌入模型
    model = SentenceTransformer('all-MPNet-base-v2')

    #生成随机文本
    sentences = generate_random_sentences()

    #将文本转为向量并归一化
    embeddings = model.encode(sentences).astype('float32')
    normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    #创建余弦相似度索引（内积）
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(normalized_embeddings)

    #查询相似文本
    query_vector = model.encode([query_sentence]).astype('float32')
    normalized_query = query_vector / np.linalg.norm(query_vector, axis=1, keepdims=True)

    D, I = index.search(normalized_query, k=4)  #查找最相似的3个文本(除去原有的数据库中的句子)

    flag_is_plagiarism = False
    plagiarismed_sentence = "No plagiarism"

    print("Query:", query_sentence)
    print("Most similar sentences:")

    #这部分修改了
    for i, idx in enumerate(I[0]):
        if i == 0:
          continue
        if i == 1:
          if D[0][i]>= 0.5: #当最相似的那个文本的余弦相似度超过了0.5，认为存在抄袭
            flag_is_plagiarism = True
            plagiarismed_sentence = sentences[idx]
        print(f"{i}. {sentences[idx]} (Similarity: {D[0][i]:.4f})")
    return flag_is_plagiarism,plagiarismed_sentence

#添加一个函数，返回最有可能被抄袭的部分
def plagiarismed_part(plagiarismed_sentence):
    #核心思想：将文本内容以句号为分界线进行分割，从而将一个长句子拆分成多个独立的短句子。
    #我们现在有两个文本，分别是“查询文本”和“被抄袭文本”，为了方便讨论，我们将其分别命名为文本a和文本b。
    #把文本a和文本b分别根据句号为分界线分成多个句子
    #我们采用双重遍历的方法，对文本a中分割出来的每一个句子进行遍历，然后逐个去匹配文本b中的句子
    #这相当于将原有的sentences集合转化为由文本B构成的数据库
    #在每一个循环中，把原来的query_sentence换成了每一个文本a的句子
    sentences = split_sentences(plagiarismed_sentence)

    #加载更强大的句子嵌入模型
    model = SentenceTransformer('all-MPNet-base-v2')

    #将文本转为向量并进行归一化
    embeddings = model.encode(sentences).astype('float32')
    normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    #创建Faiss索引来高效计算向量之间的余弦相似度
    dim = embeddings.shape[1]# 获取嵌入向量的维度
    index = faiss.IndexFlatIP(dim)#使用内积来表示余弦相似度
    index.add(normalized_embeddings)#将所有被抄袭句子的嵌入向量加入索引中

    #从这部分开始，使用循环遍历每一个文本a的句子，并且查询相似文本
    sentences_a = split_sentences(query_sentence)
    max_Similarity = 0 #初始化最大相似度值

    for s in sentences_a:
      #将查询句子的当前子句转换为向量并归一化
      query_vector = model.encode([s]).astype('float32')
      normalized_query = query_vector / np.linalg.norm(query_vector, axis=1, keepdims=True)

      D, I = index.search(normalized_query, k=3) #查找与当前查询子句最相似的3个句子
      for i, idx in enumerate(I[0]):
        #如果当前的相似度比之前的最大相似度还高，更新最大相似度以及对应的句子
        if max_Similarity < D[0][i]:
          max_Similarity = D[0][i]
          plagiarized_text_a = s
          plagiarized_text_b = sentences[idx]
    #返回查询句子和被抄袭句子中最可能抄袭的部分，以及它们的相似度
    return plagiarized_text_a,plagiarized_text_b,max_Similarity

#分割句子的函数
def split_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

if __name__ == '__main__':
    #接受两个参数flag_is_plagiarism和plagiarismed_sentence，前者表示是否抄袭，后者表示最可能抄袭的文本是哪个
    flag_is_plagiarism , plagiarismed_sentence = test_text_similarity()
    print("\n\n\n")
    if flag_is_plagiarism:
      print("Plagiarism has occurred！")
      print("Query:", query_sentence)
      print("Most similar sentences:", plagiarismed_sentence)
      print("\n\n\n")
      plagiarized_text_a , plagiarized_text_b , max_Similarity = plagiarismed_part(plagiarismed_sentence)
      print("The most likely plagiarized text is:")
      print(plagiarized_text_a)
      print(plagiarized_text_b)
      print("Their similarity is:",max_Similarity)
    else:
      print(plagiarismed_sentence)



Query: I enjoy playing basketball with my friends. We play every weekend.
Most similar sentences:
1. He enjoy playing basketball with my friends. They play every weekend. (Similarity: 0.5823)
2. Friendship makes life more enjoyable. It adds meaning to our lives. (Similarity: 0.3458)
3. I am learning to play the guitar. It's a challenging but rewarding hobby. (Similarity: 0.3156)




Plagiarism has occurred！
Query: I enjoy playing basketball with my friends. We play every weekend.
Most similar sentences: He enjoy playing basketball with my friends. They play every weekend.




The most likely plagiarized text is:
We play every weekend.
They play every weekend.
Their similarity is: 0.8143205


为了代码的简洁性和可读性，我们使用了文件的读写操作，使用我们提供好的六个 data.txt 文件即可进行数据操作：

1.若是在colab云端中运行，则需要手动上传相应的六个data.txt文件到相对目录content文件夹下。

2.若本地则可以直接运行。

代码的结构和前文中给出的简单的直接操作的结构是一致的，只是数据的读写方式从直接读写变为了文件读写。

In [ ]:
import faiss
import numpy as np
import nltk
import os
import random
from sentence_transformers import SentenceTransformer


#更改：查询目标改成全局参数
query_sentence = ""

# 检查当前工作目录
print("当前工作目录:", os.getcwd())

# 列出当前目录中的所有文件
print("当前目录包含的文件:", os.listdir(os.getcwd()))

#更改：随机生成100句文本，改成双句子
def generate_sentences(plagiarism_type):
    if plagiarism_type == "no_plagiarism":
      data_file_name = "data_no_plagiarism.txt"
      test_file_name = "test_no_plagiarism.txt"
    elif plagiarism_type == "explicit_plagiarism":
      data_file_name = "data_explicit_plagiarism.txt"
      test_file_name = "test_explicit_plagiarism.txt"
    else:
      data_file_name = "data_implicit__plagiarism.txt"
      test_file_name = "test_implicit_plagiarism.txt"

    # 获取当前工作目录
    current_directory = os.getcwd()

    # 将当前目录路径与文件名组合成完整的文件路径
    data_file_path = os.path.join(current_directory, data_file_name)
    test_file_path = os.path.join(current_directory, test_file_name)

    data_sentences = []  # 初始化一个空列表，用于存储从文件中读取的句子
    test_sentences = []
    try:
        # 尝试用只读模式打开指定路径下的文件，编码方式为 UTF-8
        with open(data_file_path, 'r', encoding='utf-8') as file:
            # 使用列表推导式逐行读取文件内容，并去掉每行的首尾空白字符（如换行符和多余空格）
            data_sentences = [line.strip() for line in file.readlines()]
    except FileNotFoundError:  # 捕获文件未找到的异常
        # 如果文件不存在，打印提示信息，显示缺失的文件名及当前目录路径
        print(f"文件 {data_file_name} 不存在，请确保它位于 {current_directory} 文件夹中。")


    try:
        # 尝试用只读模式打开指定路径下的文件，编码方式为 UTF-8
        with open(test_file_path, 'r', encoding='utf-8') as file:
            # 使用列表推导式逐行读取文件内容，并去掉每行的首尾空白字符（如换行符和多余空格）
            test_sentences = [line.strip() for line in file.readlines()]
    except FileNotFoundError:  # 捕获文件未找到的异常
        # 如果文件不存在，打印提示信息，显示缺失的文件名及当前目录路径
        print(f"文件 {test_file_name} 不存在，请确保它位于 {current_directory} 文件夹中。")

    return data_sentences,test_sentences  # 返回存储句子的列表

#文本向量化并使用改进的余弦相似度
def test_text_similarity(plagiarism_type):
    #加载更强大的句子嵌入模型
    model = SentenceTransformer('all-MPNet-base-v2')

    #生成文本
    data_sentences,test_sentences = generate_sentences(plagiarism_type)

    #将文本转为向量并归一化
    data_embeddings = model.encode(data_sentences).astype('float32')
    data_normalized_embeddings = data_embeddings / np.linalg.norm(data_embeddings, axis=1, keepdims=True)

    test_embeddings = model.encode(test_sentences).astype('float32')
    test_normalized_embeddings = test_embeddings / np.linalg.norm(test_embeddings, axis=1, keepdims=True)

    #创建余弦相似度索引（内积）
    dim = data_embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(data_normalized_embeddings)

    #查询相似文本
    query_sentence = random.choice(test_sentences) # 随机的选择一个句子查询
    query_vector = model.encode([query_sentence]).astype('float32')
    normalized_query = query_vector / np.linalg.norm(query_vector, axis=1, keepdims=True)

    D, I = index.search(normalized_query, k=4)  # 查找最相似的3个文本(除去原有的数据库中的句子)

    flag_is_plagiarism = False
    plagiarismed_sentence = "No plagiarism"

    #显示查询句子和查询结果
    print("Query:", query_sentence)
    print("Most similar sentences:")

    #循环查询
    for i, idx in enumerate(I[0]):
        if i == 0:
          continue
        if i == 1:
          if D[0][i]>= 0.5: #当最相似的那个文本的余弦相似度超过了0.5，认为存在抄袭
            flag_is_plagiarism = True
            plagiarismed_sentence = data_sentences[idx]
        print(f"{i}. {data_sentences[idx]} (Similarity: {D[0][i]:.4f})")
    return flag_is_plagiarism,plagiarismed_sentence

#添加一个函数，返回最有可能被抄袭的部分
def plagiarismed_part(plagiarismed_sentence):
    #核心思想：将文本内容以句号为分界线进行分割，从而将一个长句子拆分成多个独立的短句子。
    #我们现在有两个文本，分别是“查询文本”和“被抄袭文本”，为了方便讨论，我们将其分别命名为文本a和文本b。
    #把文本a和文本b分别根据句号为分界线分成多个句子
    #我们采用双重遍历的方法，对文本a中分割出来的每一个句子进行遍历，然后逐个去匹配文本b中的句子
    #这相当于将原有的sentences集合转化为由文本B构成的数据库
    #在每一个循环中，把原来的query_sentence换成了每一个文本a的句子
    sentences = split_sentences(plagiarismed_sentence)

    #加载更强大的句子嵌入模型
    model = SentenceTransformer('all-MPNet-base-v2')

    #将文本转为向量并进行归一化
    embeddings = model.encode(sentences).astype('float32')
    normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    #创建Faiss索引来高效计算向量之间的余弦相似度
    dim = embeddings.shape[1]# 获取嵌入向量的维度
    index = faiss.IndexFlatIP(dim)#使用内积来表示余弦相似度
    index.add(normalized_embeddings)#将所有被抄袭句子的嵌入向量加入索引中

    #从这部分开始，使用循环遍历每一个文本a的句子，并且查询相似文本
    sentences_a = split_sentences(query_sentence)
    max_Similarity = 0 #初始化最大相似度值

    for s in sentences_a:
      #将查询句子的当前子句转换为向量并归一化
      query_vector = model.encode([s]).astype('float32')
      normalized_query = query_vector / np.linalg.norm(query_vector, axis=1, keepdims=True)

      D, I = index.search(normalized_query, k=3) #查找与当前查询子句最相似的3个句子
      for i, idx in enumerate(I[0]):
        #如果当前的相似度比之前的最大相似度还高，更新最大相似度以及对应的句子
        if max_Similarity < D[0][i]:
          max_Similarity = D[0][i]
          plagiarized_text_a = s
          plagiarized_text_b = sentences[idx]
    #返回查询句子和被抄袭句子中最可能抄袭的部分，以及它们的相似度
    return plagiarized_text_a,plagiarized_text_b,max_Similarity

#添加：分割句子的函数
def split_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

def main():
  plagiarism_types = ["no_plagiarism","explicit_plagiarism","implicit_plagiarism"]
  for plagiarism_type in plagiarism_types:
    flag_is_plagiarism , plagiarismed_sentence = test_text_similarity(plagiarism_type)
    print("\n\n\n")
    if flag_is_plagiarism:
      print("Plagiarism has occurred！")
      print("Query:", query_sentence)
      print("Most similar sentences:", plagiarismed_sentence)
      print("\n\n\n")
      plagiarized_text_a , plagiarized_text_b , max_Similarity = plagiarismed_part(plagiarismed_sentence)
      print("The most likely plagiarized text is:")
      print(plagiarized_text_a)
      print(plagiarized_text_b)
      print("Their similarity is:",max_Similarity)
    else:
      print(plagiarismed_sentence)

if __name__ == '__main__':
    #更改：接受两个参数 flag_is_plagiarism 和 plagiarismed_sentence ，前者表示是否抄袭，后者表示最可能抄袭的文本是哪个
    main()


